# TO DO

- [ ] add reward scaling
- [x] override train_agent_batch_with_evaluation
- [ ] support calllable object(hook)
- [ ] change original function to 
 - [ ] return float reward
 - [ ] only one Done
 - [ ] info["needs_reset"]

In [1]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import
from future import standard_library
standard_library.install_aliases()  # NOQA
import argparse
import sys
import logging
from collections import deque

import chainer
from chainer import optimizers
import gym
from gym import spaces
import gym.wrappers
import numpy as np

from chainer import cuda
import chainerrl
from chainerrl.agents.ddpg_ma import DDPG_MA
from chainerrl.agents.ddpg import DDPGModel
from chainerrl import experiments
from chainerrl import explorers
from chainerrl import misc
from chainerrl import policy
from chainerrl import q_functions
from chainerrl import replay_buffer

from flow.multiagent_envs import MultiWaveAttenuationMergePOEnvMeanRew
from flow.scenarios import MergeScenario
from flow.utils.registry import make_create_env

benchmark_name = 'multi_merge'

In [19]:
def make_env(create_env):
    def _thunk():
        env = create_env()
        return env
    return _thunk

def make_batch_env(test):
    return chainerrl.envs.MultiprocessVectorEnv(
        [make_env(create_env) for i in range(num_envs)])

In [3]:
gpu = None
seed = 0
final_exploration_steps = 10**6
actor_lr = 1e-4
critic_lr = 1e-3
steps = 10 ** 7
n_hidden_channels = 300
n_hidden_layers = 3
replay_start_size = 5000
n_update_times = 1
target_update_interval = 1
target_update_method = 'soft'
soft_update_tau = 1e-2
update_interval = 4
eval_n_runs = 100
eval_interval = 10**5
gamma = 0.995
minibatch_size = 200
use_bn = True
reward_scale_factor = 1e-2
return_window_size = 100
step_offset = 0
log_interval = 5
num_envs = 2
outdir = '../result'

In [4]:
# Set a random seed used in ChainerRL
misc.set_random_seed(seed)

def reward_filter(r):
    return r * args.reward_scale_factor

# Set different random seeds for different subprocesses.
# If seed=0 and processes=4, subprocess seeds are [0, 1, 2, 3].
# If seed=1 and processes=4, subprocess seeds are [4, 5, 6, 7].
process_seeds = np.arange(num_envs) + seed * num_envs
assert process_seeds.max() < 2 ** 32

# outdir = experiments.prepare_output_dir('result')

In [5]:
benchmark  = __import__(
    "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.mean_rew_flow_params
HORIZON = flow_params['env'].horizon

create_env, env_name = make_create_env(params=flow_params, version=0)

In [6]:
sample_env = create_env()

timestep_limit = flow_params["env"].horizon
obs_size = np.asarray(sample_env.observation_space.shape).prod()
action_space = sample_env.action_space

In [7]:
action_size = np.asarray(action_space.shape).prod()
if use_bn:
    q_func = q_functions.FCBNLateActionSAQFunction(
        obs_size, action_size,
        n_hidden_channels=n_hidden_channels,
        n_hidden_layers=n_hidden_layers,
        normalize_input=True)
    pi = policy.FCBNDeterministicPolicy(
        obs_size, action_size=action_size,
        n_hidden_channels=n_hidden_channels,
        n_hidden_layers=n_hidden_layers,
        min_action=action_space.low, max_action=action_space.high,
        bound_action=True,
        normalize_input=True)
else:
    q_func = q_functions.FCSAQFunction(
        obs_size, action_size,
        n_hidden_channels=n_hidden_channels,
        n_hidden_layers=n_hidden_layers)
    pi = policy.FCDeterministicPolicy(
        obs_size, action_size=action_size,
        n_hidden_channels=n_hidden_channels,
        n_hidden_layers=n_hidden_layers,
        min_action=action_space.low, max_action=action_space.high,
        bound_action=True)

In [8]:
model = DDPGModel(q_func=q_func, policy=pi)
opt_a = optimizers.Adam(alpha=actor_lr)
opt_c = optimizers.Adam(alpha=critic_lr)
opt_a.setup(model['policy'])
opt_c.setup(model['q_function'])
opt_a.add_hook(chainer.optimizer.GradientClipping(1.0), 'hook_a')
opt_c.add_hook(chainer.optimizer.GradientClipping(1.0), 'hook_c')

rbuf = replay_buffer.ReplayBuffer(5 * 10 ** 5)

In [9]:
ou_sigma = (action_space.high - action_space.low) * 0.2
explorer = explorers.AdditiveOU(sigma=ou_sigma)
agent = DDPG_MA(model, opt_a, opt_c, rbuf, gamma=gamma,
             explorer=explorer, replay_start_size=replay_start_size,
             target_update_method=target_update_method,
             target_update_interval=target_update_interval,
             update_interval=update_interval,
             soft_update_tau=soft_update_tau,
             n_times_update=n_update_times,
             gpu=gpu, minibatch_size=minibatch_size)

In [21]:
env = make_batch_env(False)

In [22]:
experiments.evaluator.batch_run_evaluation_episodes(env, agent, 1000, None)

[99.91539795795482, 68.58812206607693, 107.17406151588956]

In [23]:
# from IPython.core.debugger import Pdb; Pdb().set_trace()

eval_stats = experiments.eval_performance(
    env=env,
    agent=agent,
    n_steps=None,
    n_episodes=2,
    max_episode_len=timestep_limit)
print('n_runs: {} mean: {} median: {} stdev {}'.format(
    eval_n_runs, eval_stats['mean'], eval_stats['median'],
    eval_stats['stdev']))

n_runs: 100 mean: 53.39794724552133 median: 53.39794724552133 stdev 75.51610119749932


1. experiments.train_agent_batch_with_evaluation()

2. contents in train_agent_batch

    1. actions = agent.batch_act_and_train(obss)
    2. env.step(actions)
    3. compute resets
    4. agent.batch_observe_and_train(obss, rs, dones, resets): in this env, done==True when horizon is reached. So resets is same as dones

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
recent_returns = deque(maxlen=return_window_size)

num_envs = env.num_envs
episode_r = np.zeros(num_envs, dtype=np.float64)
episode_idx = np.zeros(num_envs, dtype='i')
episode_len = np.zeros(num_envs, dtype='i')

# o_0, r_0
obss = env.reset()
rs = np.zeros(num_envs, dtype='f')

t = step_offset

In [14]:
for _ in range(5):
    # a_t
    actions = agent.batch_act_and_train(obss)
    # o_{t+1}, r_{t+1}
    obss, rs, dones, infos = env.step(actions)
    episode_r += rs
    episode_len += 1

    # mask for reset the env(when collision or horizon)
    env_reset = dones
    not_env_reset = np.logical_not(env_reset)  # doesn't reset when True

    # Agent observes the consequences
    agent.batch_observe_and_train(obss, rs, dones, env_reset)

    episode_idx += env_reset
    recent_returns.extend(episode_r[env_reset])

    for _ in range(num_envs):
        t += 1

    # logger should be here
    if (log_interval is not None
            and t >= log_interval
            and t % log_interval < num_envs):
        logger.info(
            'outdir:{} step:{} episode:{} last_R: {} average_R:{}'.format(  # NOQA
                outdir,
                t,
                np.sum(episode_idx),
                recent_returns[-1] if recent_returns else np.nan,
                np.mean(recent_returns) if recent_returns else np.nan,
            ))
        logger.info('statistics: {}'.format(agent.get_statistics()))

    # evaluator should be here

    if t >= steps:
        break

    # Start new episodes if needed
    episode_r[env_reset] = 0
    episode_len[env_reset] = 0
    obss = env.reset(not_env_reset)

DEBUG:chainerrl.agents.ddpg:t:0 a:[0.17891979] q:[[-0.37691563]
 [-0.29338697]
 [-0.1590812 ]]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.5457015]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.2952096]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.6980542]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.1124804]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[0.69293666]
DEBUG:chainerrl.agents.ddpg:t:4 a:[0.09080206] q:[[-0.29664254]
 [-0.1683121 ]]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[1.1955103]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[2.1975727]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[1.6391486]
DEBUG:chainerrl.explorers.additive_ou:t:4 noise:[1.3692688]
DEBUG:chainerrl.agents.ddpg:t:8 a:[0.09270164] q:[[-0.29954857]
 [-0.17745703]]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[1.4674435]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[1.4067122]
DEBUG:chainerrl.explorers.additive_ou:t:8 noise:[0.76240957]
DEBUG:chainerrl.explore

# For not native multi-merge env

In [ ]:
class DDPG_MA(ddpg.DDPG):
    def batch_act(self, batch_obs):
        """Select a batch of actions for evaluation.
        Args:
            batch_obs (Sequence of ~object): Observations.
        Returns:
            Sequence of ~object: Actions.
        """

        batch_actions = []
        for env_obs in batch_obs:
            keys, obss = list(env_obs.keys()), list(env_obs.values())
            obss = [obs.astype(np.float32) for obs in obss]
            
            with chainer.using_config('train', False), chainer.no_backprop_mode():
                batch_xs = self.batch_states(obss, self.xp, self.phi)
                batch_action = self.policy(batch_xs).sample()
                # Q is not needed here, but log it just for information
                q = self.q_function(batch_xs, batch_action)

            batch_actions.append({key:cuda.to_cpu(action.array) for key, action in zip(keys, batch_action)})
            
            # Update stats
            self.average_q *= self.average_q_decay
            self.average_q += (1 - self.average_q_decay) * float(
                q.array.mean(axis=0))
            
        self.logger.debug('t:%s a:%s q:%s',
                          self.t, batch_action.array[0], q.array)
        return batch_actions

    def batch_act_and_train(self, batch_obs):
        """Select a batch of actions for training.
        Args:
            batch_obs (Sequence of ~object): Observations.
        Returns:
            Sequence of ~object: Actions.
        """

        batch_greedy_action = self.batch_act(batch_obs)
        batch_action = [
            {key: self.explorer.select_action(self.t, lambda: action) for key, action in actions.items()}
            for actions in batch_greedy_action] 

        self.batch_last_obs = list(batch_obs)
        self.batch_last_action = list(batch_action)
        
        return batch_action

    def batch_observe_and_train(
            self, batch_obs, batch_reward, batch_done, batch_reset):
        """Observe a batch of action consequences for training.
        Args:
            batch_obs (Sequence of ~object): Observations.
            batch_reward (Sequence of float): Rewards.
            batch_done (Sequence of boolean): Boolean values where True
                indicates the current state is terminal.
            batch_reset (Sequence of boolean): Boolean values where True
                indicates the current episode will be reset, even if the
                current state is not terminal.
        Returns:
            None
        """
        
        for i in range(len(batch_obs)):
            if self.batch_last_obs[i] is not None:
                assert self.batch_last_action[i] is not None           
                for key in self.batch_last_obs[i].keys():
                    if key in batch_obs[i]:
                        self.t += 1
                        # Update the target network
                        if self.t % self.target_update_interval == 0:
                            self.sync_target_network()
                        # Add a transition to the replay buffer
                        self.replay_buffer.append(
                            state=self.batch_last_obs[i][key],
                            action=self.batch_last_action[i][key],
                            reward=batch_reward[i][key],
                            next_state=batch_obs[i][key],
                            next_action=None,
                            is_state_terminal=batch_done[i][key]
                        )
                        self.replay_updater.update_if_necessary(self.t)
                # set None when env is beggining
                if batch_reset[i]:
                    self.batch_last_obs[i] = None

In [14]:
for _ in range(5):
    # a_t
    actions = agent.batch_act_and_train(obss)
    # o_{t+1}, r_{t+1}
    obss, rs, dones, infos = env.step(actions)
    episode_r += [np.mean(list(rs[i].values())) for i in range(len(rs))]
    episode_len += 1

    # mask for reset the env(when collision or horizon)
    env_reset = [done["__all__"] for done in dones]
    not_env_reset = np.logical_not(env_reset)  # doesn't reset when True

    # Agent observes the consequences
    agent.batch_observe_and_train(obss, rs, dones, env_reset)

    episode_idx += env_reset
    recent_returns.extend(episode_r[env_reset])

    for _ in range(num_envs):
        t += 1

    # logger should be here
    if (log_interval is not None
            and t >= log_interval
            and t % log_interval < num_envs):
        logger.info(
            'outdir:{} step:{} episode:{} last_R: {} average_R:{}'.format(  # NOQA
                outdir,
                t,
                np.sum(episode_idx),
                recent_returns[-1] if recent_returns else np.nan,
                np.mean(recent_returns) if recent_returns else np.nan,
            ))
        logger.info('statistics: {}'.format(agent.get_statistics()))

    # evaluator should be here

    if t >= steps:
        break

    # Start new episodes if needed
    episode_r[env_reset] = 0
    episode_len[env_reset] = 0
    obss = env.reset(not_env_reset)

DEBUG:chainerrl.agents.ddpg:t:0 a:[0.08707273] q:[[-0.2751863 ]
 [-0.15872331]]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.5457015]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.2952096]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.6980542]
DEBUG:chainerrl.explorers.additive_ou:t:0 noise:[1.1124804]


AttributeError: 'numpy.float64' object has no attribute 'values'